In [34]:
!pip install groq duckduckgo-search

In [35]:
!pip install -U agno

# Basic Agent & Agent with Tools

In [36]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.models.groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from google.colab import userdata
import os

# Install ddgs if not already installed
try:
    from ddgs import DDGS
except ImportError:
    !pip install ddgs


# os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API")


agent=Agent(
    model=Groq(id="openai/gpt-oss-20b"),
    description="You are an assistant please reply based on the question",
    tools=[DuckDuckGoTools()],
    markdown=True
)

# agent.print_response("Who won the India vs Newzealand finals in CT 2025")

agent.print_response(
        "Final match details of world cup 2011 & who lifted the WC2025?",
        stream=True,
        show_full_reasoning=True,
        stream_intermediate_steps=True,
    )

Output()

In [ ]:
from agno.playground import Playground, serve_playground_app
playground = Playground(agents=[agent]).get_app()

# Agent with Instruction

In [12]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.models.groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from google.colab import userdata

# os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API")


agent=Agent(
    model=Groq(id="openai/gpt-oss-20b"),
    description="You are an assistant please reply based on the question",
    tools=[DuckDuckGoTools()],
    instructions=[
        "Use some detailed statistics information for clarity",
        "Also include the score details along with man of match informations",
    ],
    markdown=True
)

agent.print_response(
        "Final match details of world cup 2011 & who lifted the WC2025?",
        stream=True,
        show_full_reasoning=True,
        stream_intermediate_steps=True,
    )

Output()

# Agent with Reasoning - Think & Analyze before deliver

In [14]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.models.groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from google.colab import userdata
from agno.tools.reasoning import ReasoningTools


# os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API")


agent=Agent(
    model=Groq(id="openai/gpt-oss-20b"),
    description="You are an assistant please reply based on the question",
    tools=[ReasoningTools(add_instructions=True),DuckDuckGoTools()],
    instructions=[
        "Use some detailed statistics information for clarity",
        "Also include the score details along with man of match informations",
    ],
    markdown=True
)

agent.print_response(
        "Final match details of world cup 2011 & who lifted the WC2025?",
        stream=True,
        show_full_reasoning=True,
        stream_intermediate_steps=True,
    )

Output()

# Agents with Knowledge Base

In [37]:
!pip install lancedb
!pip install tantivy
!pip install pylance
!pip install ChromaDB

In [38]:
from agno.agent import Agent
# from agno.embedder.openai import OpenAIEmbedder # This import seems incorrect
from agno.knowledge.remote_content import UrlKnowledge
# from agno.models.anthropic import Claude
from agno.tools.reasoning import ReasoningTools
from agno.vectordb.lancedb import LanceDb, SearchType
from agno.models.google import Gemini
from google.colab import userdata
import os
# os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API")
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API")


from agno.models.google import GeminiEmbedder # Assuming embedders are now under models.google


# Embed sentence in database
embeddings = GeminiEmbedder().get_embedding("The quick brown fox jumps over the lazy dog.")

# Print the embeddings and their dimensions
print(f"Embeddings: {embeddings[:5]}")
print(f"Dimensions: {len(embeddings)}")

# Access UrlKnowledge using the imported module (assuming it exists within remote_content)
# UrlKnowledge = agno.knowledge.remote_content.UrlKnowledge # This line is commented out as we're only fixing the import for now

ImportError: cannot import name 'UrlKnowledge' from 'agno.knowledge.remote_content' (/usr/local/lib/python3.12/dist-packages/agno/knowledge/remote_content/__init__.py)

In [28]:
import agno.knowledge.remote_content
print(dir(agno.knowledge.remote_content))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'remote_content']


In [25]:
import agno.models
print(dir(agno.models))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'base', 'google', 'groq', 'message', 'metrics', 'openai', 'response']


In [31]:
import agno.knowledge
print(dir(agno.knowledge))

['Knowledge', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'chunking', 'content', 'document', 'embedder', 'knowledge', 'reader', 'remote_content', 'reranker', 'types']


In [18]:
import agno
print(dir(agno))

['PackageNotFoundError', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'agent', 'api', 'cloud', 'db', 'exceptions', 'knowledge', 'media', 'memory', 'models', 'reasoning', 'run', 'session', 'tools', 'utils', 'version']


In [32]:
# Load Agno documentation in a knowledge base - LanceDB
knowledge = UrlKnowledge(
    urls=["https://docs.agno.com/introduction/agents.md"],
    vector_db=LanceDb(
        uri="tmp/lancedb",
        table_name="agno_docs",
        search_type=SearchType.hybrid,
        # Use Gemini for embeddings; since it's free!
        embedder=GeminiEmbedder(),
    ),
)

'''
# Load Agno documentation in a knowledge base - ChromaDB

from agno.vectordb.chroma import ChromaDb

chroma_db = ChromaDb(collection="recipes", path="tmp/chromadb", persistent_client=True, embedder=GeminiEmbedder())

knowledge = UrlKnowledge(
    urls=["https://docs.agno.com/introduction/agents.md"],
    vector_db=chroma_db,
)
'''

NameError: name 'UrlKnowledge' is not defined

In [ ]:
agent = Agent(
    name="Agno Assist",
    model=Gemini(id="gemini-2.0-flash"),
    instructions=[
        "Use tables to display data.",
        "Include sources in your response.",
        "Search your knowledge before answering the question.",
        "Only include the output in your response. No other text.",
    ],
    knowledge=knowledge,
    tools=[ReasoningTools(add_instructions=True)],
    add_datetime_to_instructions=True,
    markdown=True,
)

In [ ]:
if __name__ == "__main__":
    # Load the knowledge base, comment out after first run
    # Set recreate to True to recreate the knowledge base if needed
    agent.knowledge.load(recreate=False)
    agent.print_response(
        "What are Agents?",
        stream=True,
        show_full_reasoning=True,
        stream_intermediate_steps=True,
    )

INFO Loading knowledge base

INFO Reading: https://docs.agno.com/introduction/agents.md

INFO Skipped 4 existing/duplicate documents.

INFO Added 0 documents to knowledge base

Output()

INFO Found 4 documents

# Agents with Storage & Memory - Sqlite Storage

In [1]:
from agno.agent import Agent
# from agno.models.anthropic import Claude
from agno.storage.sqlite import SqliteStorage
from agno.tools.duckduckgo import DuckDuckGoTools
from rich.pretty import pprint
from agno.models.google import Gemini
from google.colab import userdata
import os
from agno.tools.reasoning import ReasoningTools

os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API")

agent = Agent(
    # This session_id is usually auto-generated
    # But for this example, we can set it to a fixed value
    # This session will now forever continue as a very long chat
    session_id="agent_session_which_is_autogenerated_if_not_set",
    model=Gemini(id="gemini-2.0-flash"),
    storage=SqliteStorage(table_name="agent_sessions", db_file="tmp/agents.db"),
    tools=[DuckDuckGoTools()],
    add_history_to_messages=True,
    num_history_runs=3,
    add_datetime_to_instructions=True,
    markdown=True,
)

if __name__ == "__main__":
    print(f"Session id: {agent.session_id}")
    # agent.print_response("How many people live in Canada?")
    # agent.print_response("What is their national anthem?")
    # agent.print_response("List my messages one by one")
    agent.print_response(
        "How many people live in Canada?",
        stream=True,
        show_full_reasoning=True,
        stream_intermediate_steps=True,
    )
    agent.print_response(
        "What is their national anthem?",
        stream=True,
        show_full_reasoning=True,
        stream_intermediate_steps=True,
    )
    agent.print_response(
        "List my messages one by one",
        stream=True,
        show_full_reasoning=True,
        stream_intermediate_steps=True,
    )

    # Print all messages in this session
    messages_in_session = agent.get_messages_for_session()
    pprint(messages_in_session)

ModuleNotFoundError: No module named 'agno'